In [1]:
import os, sys
import numpy as np
sys.path.append('../')

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
from onsstove.onsstove import OnSSTOVE
from onsstove.layer import RasterLayer, VectorLayer

In [4]:
nepal = OnSSTOVE(project_crs=3857, cell_size=(1000, 1000))
output_directory = 'nepal'
nepal.output_directory = output_directory

In [5]:
path = r"data\population_npl_2018-10-01.tif"
os.makedirs('nepal/rasters/population', exist_ok=True)
population = RasterLayer('rasters', 'population', layer_path=path, resample='sum')
population.reproject(3857, 'nepal/rasters/population', 1000, 1000)
nepal.base_layer = population
nepal.population_to_dataframe(population)

In [6]:
np.nansum(population.layer)

28513543.73833791

In [7]:
path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Administrative\npl_admbnda_nd_20201117_shp\npl_admbnda_adm0_nd_20201117.shp"
mask_layer = VectorLayer('vector', 'adm_0', layer_path=path)
os.makedirs('nepal/vector/adm_0', exist_ok=True)
mask_layer.reproject(3857, 'nepal/vector/adm_0')
nepal.mask_layer = mask_layer
# add admin_1 or 2 or 3 layer to gdf here

In [8]:
path = r"C:\Users\camilorg\Box Sync\Clean cooking Africa paper\05. OnSSTOVE test case\GQ_test_file_social_specs.csv"
nepal.read_scenario_data(path)

In [9]:
path = r"C:\Users\camilorg\Box Sync\Clean cooking Africa paper\05. OnSSTOVE test case\GQ_test_file_tech_specs.csv"
nepal.read_tech_data(path)

In [10]:
rr_0_alri, rr_0_copd, rr_0_ihd, rr_0_lc = nepal.base_fuel.relative_risk()

In [11]:
print(rr_0_alri, rr_0_copd, rr_0_ihd, rr_0_lc)

3.379491698111922 2.6016762384532757 2.9461880136695875 3.807237414338239


In [12]:
paf_0_alri = nepal.base_fuel.paf(rr_0_alri, 1 - nepal.specs['clean_cooking_access'])
paf_0_copd = nepal.base_fuel.paf(rr_0_copd, 1 - nepal.specs['clean_cooking_access'])
paf_0_lc  = nepal.base_fuel.paf(rr_0_ihd, 1 - nepal.specs['clean_cooking_access'])
paf_0_ihd = nepal.base_fuel.paf(rr_0_lc, 1 - nepal.specs['clean_cooking_access'])

In [14]:
nepal.base_fuel.carb()

In [15]:
friction_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\2020_walking_only_friction_surface\2020_walking_only_friction_surface.geotiff"
forest_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\Forest cover\Global Forest Cover Change (GFCC).tif"

nepal.base_fuel.total_time(nepal.specs, friction_path, forest_path, population.path, output_directory)

In [16]:
nepal.raster_to_dataframe(nepal.base_fuel.total_time_yr, name='base_fuel_time', method='read')

In [17]:
path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Power network\Nepal_DL0\Nepal_DL0.shp"
mv_lines = VectorLayer('vector', 'MV_line', layer_path=path)
os.makedirs('nepal/vector/MV_line', exist_ok=True)
mv_lines.reproject(3857, 'nepal/vector/MV_line')

path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Power network\HV-network\HV_lines_georeferenced.shp"
hv_lines = VectorLayer('vector', 'HV_line', layer_path=path)
os.makedirs('nepal/vector/HV_line', exist_ok=True)
hv_lines.reproject(3857, 'nepal/vector/HV_line')

In [18]:
nepal.distance_to_electricity(hv_lines=hv_lines, mv_lines=mv_lines)

In [19]:
path = r"data\nighttime_lights.tif"
os.makedirs('nepal/rasters/Night_lights', exist_ok=True)
ntl = RasterLayer('rasters', 'Night_lights', layer_path=path, resample='average')
ntl.align(population.path, 'nepal/rasters/Night_lights')
ntl.layer[np.isnan(ntl.layer)] = 0 # hey lets make sure that we interpolate NaNs

nepal.raster_to_dataframe(ntl.layer, name='Night_lights', method='read')

In [20]:
nepal.gdf

,geometry,Pop,base_fuel_time,HV_line_dist,MV_line_dist,Night_lights
0,POINT (9090104.530 3543192.836),31.171455,1365.467180,58.872746,40.521599,0.000000
1,POINT (9090104.530 3536192.836),46.757183,1429.499906,52.392746,33.837849,0.000000
2,POINT (9016104.530 3535192.836),13.789093,1259.615643,NaN,NaN,0.000000
3,POINT (9083104.530 3535192.836),5.517799,1257.989529,54.708317,35.846897,0.000000
4,POINT (9084104.530 3535192.836),15.585728,1263.917683,54.203320,35.355339,0.000000
...,...,...,...,...,...,...
97088,POINT (9800104.530 3044192.836),3.297608,1477.709497,NaN,NaN,0.000000
97089,POINT (9721104.530 3043192.836),49.452458,2075.862576,NaN,NaN,0.000000
97090,POINT (9722104.530 3043192.836),642.881959,2067.246934,11.000000,0.000000,0.891932
97091,POINT (9723104.530 3043192.836),134.605801,2058.631292,12.000000,1.000000,0.806685


In [21]:
nepal.calibrate_pop()

In [22]:
nepal.calibrate_urban()

In [23]:
nepal.gdf

,geometry,Pop,base_fuel_time,HV_line_dist,MV_line_dist,Night_lights,Calibrated_pop,IsUrban
0,POINT (9090104.530 3543192.836),31.171455,1365.467180,58.872746,40.521599,0.000000,31.276903,0
1,POINT (9090104.530 3536192.836),46.757183,1429.499906,52.392746,33.837849,0.000000,46.915354,0
2,POINT (9016104.530 3535192.836),13.789093,1259.615643,NaN,NaN,0.000000,13.835740,0
3,POINT (9083104.530 3535192.836),5.517799,1257.989529,54.708317,35.846897,0.000000,5.536465,0
4,POINT (9084104.530 3535192.836),15.585728,1263.917683,54.203320,35.355339,0.000000,15.638451,0
...,...,...,...,...,...,...,...,...
97088,POINT (9800104.530 3044192.836),3.297608,1477.709497,NaN,NaN,0.000000,3.308763,0
97089,POINT (9721104.530 3043192.836),49.452458,2075.862576,NaN,NaN,0.000000,49.619747,0
97090,POINT (9722104.530 3043192.836),642.881959,2067.246934,11.000000,0.000000,0.891932,645.056715,1
97091,POINT (9723104.530 3043192.836),134.605801,2058.631292,12.000000,1.000000,0.806685,135.061148,0


In [24]:
nepal.elec_current()

We have identified the existence of transformers or MV lines as input data; therefore we proceed using those for the calibration


In [25]:
nepal.gdf

,geometry,Pop,base_fuel_time,HV_line_dist,MV_line_dist,Night_lights,Calibrated_pop,IsUrban,Elec_pop_calib,Current_elec,Elec_dist,Final_Elec_Code2020
0,POINT (9090104.530 3543192.836),31.171455,1365.467180,58.872746,40.521599,0.000000,31.276903,0,0.0,0,40.521599,99
1,POINT (9090104.530 3536192.836),46.757183,1429.499906,52.392746,33.837849,0.000000,46.915354,0,0.0,0,33.837849,99
2,POINT (9016104.530 3535192.836),13.789093,1259.615643,NaN,NaN,0.000000,13.835740,0,0.0,0,NaN,99
3,POINT (9083104.530 3535192.836),5.517799,1257.989529,54.708317,35.846897,0.000000,5.536465,0,0.0,0,35.846897,99
4,POINT (9084104.530 3535192.836),15.585728,1263.917683,54.203320,35.355339,0.000000,15.638451,0,0.0,0,35.355339,99
...,...,...,...,...,...,...,...,...,...,...,...,...
97088,POINT (9800104.530 3044192.836),3.297608,1477.709497,NaN,NaN,0.000000,3.308763,0,0.0,0,NaN,99
97089,POINT (9721104.530 3043192.836),49.452458,2075.862576,NaN,NaN,0.000000,49.619747,0,0.0,0,NaN,99
97090,POINT (9722104.530 3043192.836),642.881959,2067.246934,11.000000,0.000000,0.891932,645.056715,1,NaN,1,0.000000,1
97091,POINT (9723104.530 3043192.836),134.605801,2058.631292,12.000000,1.000000,0.806685,135.061148,0,NaN,1,1.000000,1


In [ ]:
# nepal.gdf.crs = 3857
# nepal.gdf.to_file('nepal/results.gpkg', driver='GPKG')

In [26]:
nepal.gdf['Elec_pop_calib'].sum() / nepal.gdf['Calibrated_pop'].sum()

0.5189724919639153

In [27]:
nepal.techs['Electricity'].morbidity(nepal.specs, paf_0_alri, paf_0_copd, paf_0_lc, paf_0_ihd)

In [28]:
nepal.techs['Electricity'].urban_morbidity

341482.9650891477

In [29]:
nepal.techs['Electricity'].rural_morbidity

374317.8655784887

In [30]:
nepal.techs['Electricity'].mortality(nepal.specs, paf_0_alri, paf_0_copd, paf_0_lc, paf_0_ihd)

In [33]:
nepal.techs['Electricity'].carbon_emissions(nepal.specs, nepal.base_fuel.carbon)
nepal.techs['Electricity'].decreased_carbon_emissions

197562.7366143861

In [34]:
path = r"C:\Users\camilorg\Box Sync\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Demand\Wealth Index\Wealth index 2011.tif"
wealth_index = RasterLayer('rasters', 'Wealth_index', layer_path=path,  resample='average')
os.makedirs('nepal/rasters/Wealth_index', exist_ok=True)
wealth_index.align(population.path, 'nepal/rasters/Wealth_index')

In [35]:
nepal.get_value_of_time(wealth_index)

In [36]:
nepal.techs['Electricity'].time_save(nepal.gdf, nepal.specs)

In [37]:
nepal.techs['Electricity'].time_value

0         767.731520
1         861.639174
2                NaN
3         625.581760
4         633.581427
            ...     
97088            NaN
97089            NaN
97090            NaN
97091    3036.595898
97092            NaN
Length: 97093, dtype: float64